# Correlation Matrix with Pytorch

In [1]:
%load_ext autoreload

In [2]:
import os
import sys
print(os.listdir('../'))

['pyproject.toml', 'report', 'weights', 'notebooks', 'requirements.txt', 'data', '.gitignore', 'README.md', 'Makefile', 'parkinson', '.git']


In [ ]:
import sys
# sys.path.append('/home/labic/merlin_codes/dl/Parkinson-Diagnosis-Deeplearning')
sys.path.append('../')

import os
import time
    
import parkinson

import torch
from torch import nn

import pandas as pd
import networkx as nx
import numpy as np

import pickle
from joblib import Parallel, delayed
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import RandomOverSampler

In [ ]:
RDN = 50
N_CLASSES = 2
BATCH_SIZE = 32
N_EPOCHS = 200
PATIENCE = 20
LR = 0.0001
CORR_METHOD = "sliding_window"  # método de matriz de correlação: dtw, pearson, spearman, icoh (no graph.py)

## Data Processing
### Reading

In [ ]:
parkinson_data = parkinson.utils.data.batch_read('../data/PDs_columns')
control_data = parkinson.utils.data.batch_read('../data/Controls_columns')

control_atlas_data = parkinson.utils.data.select_atlas_columns(control_data, 'AAL3')
parkinson_atlas_data = parkinson.utils.data.select_atlas_columns(parkinson_data, 'AAL3')

total de pacientes: 219

In [ ]:
parkinson_correlation_matrix = parkinson.utils.graph.compute_correlation_matrix(
    parkinson_atlas_data, method=CORR_METHOD, group='parkinson'
)

In [ ]:
control_correlation_matrix = parkinson.utils.graph.compute_correlation_matrix(
    control_atlas_data, method=CORR_METHOD, group='control'
)

In [ ]:
X = parkinson.utils.data.concatenate_data(parkinson_correlation_matrix, control_correlation_matrix)
y = parkinson.utils.data.concatenate_data([1 for _ in range(len(parkinson_data))], [0 for _ in range(len(control_data))])

X, y = parkinson.utils.data.filter_data(X, y)

### Split  
- 60% treino
- 20% validação
- 20% teste

In [ ]:
X_trainval, X_test, y_trainval, y_test = train_test_split(X, y, test_size=0.2, random_state=RDN, stratify=y, shuffle=True)
X_train, X_val, y_train, y_val = train_test_split(X_trainval, y_trainval, test_size=0.25, random_state=RDN, stratify=y_trainval, shuffle=True)

ros = RandomOverSampler(random_state=RDN)
X_train, y_train = ros.fit_resample(X_train, y_train)

### Dataloaders

In [ ]:
train_loader = parkinson.utils.data.get_torch_dataloader(X_train, y_train, batch_size=BATCH_SIZE)
val_loader = parkinson.utils.data.get_torch_dataloader(X_val, y_val, batch_size=BATCH_SIZE)
test_loader = parkinson.utils.data.get_torch_dataloader(X_test, y_test, batch_size=BATCH_SIZE)

## Training

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = parkinson.NetworkModels.SimpleMLP(input_dim=X_train.shape[1], hidden_dim=16, output_dim=2)
class_weights = parkinson.utils.data.get_torch_class_weights(y_train)

In [ ]:
model.train()
criterion = nn.CrossEntropyLoss(weight=class_weights.to(device))

In [ ]:
%%time
out = parkinson.utils.train.train(model, train_loader, val_loader, class_weights, device, N_EPOCHS,  PATIENCE,LR)

In [ ]:
metrics = parkinson.utils.train.evaluate(model, test_loader, device)
print('Metrics:', metrics)
print('Done.')

In [ ]:
parkinson.utils.results.plot_losses(out['train_loss'], out['val_loss'])